In [13]:
import pyspark
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col

spark = SparkSession.builder.getOrCreate()

data_without_header = spark.read.option("inferSchema",True).option("header",False).csv("covtype.data")
print(data_without_header.summary)
colnames = ["Elevation","Aspect","Slope","Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology",\
           "Horizontal_Distance_To_Roadways","Hillshade_9am","Hillshade_noon","Hillshade_3pm",\
           "Horizontal_Distance_To_Fire_Points"] + \
           [f"Wilderness_Area_{i}" for i in range(4)] + [f"Soil_Type_{i}" for i in range(40)] + ["Cover_Type"]

data_without_header.printSchema()
data = data_without_header.toDF(*colnames).withColumn("Cover_Type",col("Cover_Type").cast(DoubleType()))
print(data.summary)
train_data,test_data = data.randomSplit([0.9,0.1])

<bound method DataFrame.summary of DataFrame[_c0: int, _c1: int, _c2: int, _c3: int, _c4: int, _c5: int, _c6: int, _c7: int, _c8: int, _c9: int, _c10: int, _c11: int, _c12: int, _c13: int, _c14: int, _c15: int, _c16: int, _c17: int, _c18: int, _c19: int, _c20: int, _c21: int, _c22: int, _c23: int, _c24: int, _c25: int, _c26: int, _c27: int, _c28: int, _c29: int, _c30: int, _c31: int, _c32: int, _c33: int, _c34: int, _c35: int, _c36: int, _c37: int, _c38: int, _c39: int, _c40: int, _c41: int, _c42: int, _c43: int, _c44: int, _c45: int, _c46: int, _c47: int, _c48: int, _c49: int, _c50: int, _c51: int, _c52: int, _c53: int, _c54: int]>
root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = 

In [14]:
from pyspark.ml.feature import VectorAssembler

input_cols = colnames[:-1]
vector_assembler = VectorAssembler(inputCols = input_cols,outputCol = "featureVector")
assembled_train_data = vector_assembler.transform(train_data)
assembled_train_data.select("featureVector").show(truncate=False)

+-----------------------------------------------------------------------------------------------------+
|featureVector                                                                                        |
+-----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0])  |
|(54,[0,1,2,5,6,7,8,9,13,18],[1874.0,18.0,14.0,90.0,208.0,209.0,135.0,793.0,1.0,1.0])                 |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1879.0,28.0,19.0,30.0,12.0,95.0,209.0,196.0,117.0,778.0,1.0,1.0])   |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1888.0,33.0,22.0,150.0,46.0,108.0,209.0,185.0,103.0,735.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1889.0,28.0,22.0,150.0,23.0,120.0,205.0,185.0,108.0,759.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1889.0,353.0,30.0,95.0,39.0,67.0,153.0,172.0,146.0,600.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1896.0,337.0,12.0,30.0,6.0,175

In [15]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(seed=1234,labelCol="Cover_Type",featuresCol="featureVector",predictionCol="prediction")
model=classifier.fit(assembled_train_data)
print(model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_39f8f47f035d, depth=5, numNodes=39, numClasses=8, numFeatures=54
  If (feature 0 <= 3047.5)
   If (feature 0 <= 2499.5)
    If (feature 3 <= 15.0)
     If (feature 12 <= 0.5)
      If (feature 23 <= 0.5)
       Predict: 4.0
      Else (feature 23 > 0.5)
       Predict: 3.0
     Else (feature 12 > 0.5)
      Predict: 6.0
    Else (feature 3 > 15.0)
     If (feature 16 <= 0.5)
      Predict: 3.0
     Else (feature 16 > 0.5)
      If (feature 9 <= 1317.5)
       Predict: 3.0
      Else (feature 9 > 1317.5)
       Predict: 4.0
   Else (feature 0 > 2499.5)
    If (feature 17 <= 0.5)
     If (feature 15 <= 0.5)
      Predict: 2.0
     Else (feature 15 > 0.5)
      Predict: 3.0
    Else (feature 17 > 0.5)
     If (feature 0 <= 2711.5)
      Predict: 3.0
     Else (feature 0 > 2711.5)
      If (feature 5 <= 1235.5)
       Predict: 5.0
      Else (feature 5 > 1235.5)
       Predict: 2.0
  Else (feature 0 > 3047.5)
   If (feature 0 <= 3

In [17]:
import pandas as pd
pd.DataFrame(model.featureImportances.toArray(),index=input_cols,columns=['importance']).sort_values(by="importance",ascending=False)

,importance
Elevation,0.831218
Soil_Type_3,0.037130
Soil_Type_1,0.032074
Hillshade_noon,0.027063
Horizontal_Distance_To_Hydrology,0.023290
Soil_Type_31,0.018641
Wilderness_Area_2,0.016227
Horizontal_Distance_To_Roadways,0.004577
Soil_Type_2,0.003711
Hillshade_9am,0.002755


In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions = model.transform(assembled_train_data)
predictions.select("Cover_Type","prediction","probability").show(10,truncate=False)

evaluator=MulticlassClassificationEvaluator(labelCol="Cover_Type",predictionCol="prediction")
evaluator.setMetricName("accuracy").evaluate(predictions)
evaluator.setMetricName("f1").evaluate(predictions)

+----------+----------+------------------------------------------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                                                         |
+----------+----------+------------------------------------------------------------------------------------------------------------------------------------+
|6.0       |3.0       |[0.0,3.158559696778269E-5,0.06620341124447252,0.6085596967782692,0.020341124447252053,0.0014529374605180038,0.30341124447252055,0.0]|
|6.0       |4.0       |[0.0,0.0,0.035739814152966405,0.24017155110793423,0.6211579699785561,0.007862759113652609,0.09506790564689063,0.0]                  |
|6.0       |3.0       |[0.0,3.158559696778269E-5,0.06620341124447252,0.6085596967782692,0.020341124447252053,0.0014529374605180038,0.30341124447252055,0.0]|
|6.0       |3.0       |[0.0,3.158559696778269E-5,0.0662034

0.6865613100558249

In [19]:
confusion_matrix = predictions.groupBy("Cover_Type").\
pivot("prediction", range(1,8)).count().\
na.fill(0.0).\
orderBy("Cover_Type")
confusion_matrix.show()

+----------+------+------+-----+----+---+---+-----+
|Cover_Type|     1|     2|    3|   4|  5|  6|    7|
+----------+------+------+-----+----+---+---+-----+
|       1.0|126132| 59039|  103|   0| 23|  8| 5371|
|       2.0| 49130|200683| 3638|  50|359| 54|  789|
|       3.0|     0|  3894|27755| 346| 19|121|    0|
|       4.0|     0|     3| 1325|1122|  0|  0|    0|
|       5.0|     0|  7821|  322|  11|434|  0|    0|
|       6.0|     0|  4438|10578| 133|  8|526|    0|
|       7.0|  7866|   235|    0|   0|  0|  0|10392|
+----------+------+------+-----+----+---+---+-----+



In [20]:
from pyspark.sql import DataFrame
def class_probabilities(data):
    total = data.count()
    return data.groupBy("Cover_Type").count().\
    orderBy("Cover_Type").\
    select(col("count").cast(DoubleType())).\
    withColumn("count_proportion", col("count")/total).\
    select("count_proportion").collect()
train_prior_probabilities = class_probabilities(train_data)
test_prior_probabilities = class_probabilities(test_data)
train_prior_probabilities

[Row(count_proportion=0.364770970753432),
 Row(count_proportion=0.4872572351203685),
 Row(count_proportion=0.06147556664268989),
 Row(count_proportion=0.0046869500007652165),
 Row(count_proportion=0.016429194533294562),
 Row(count_proportion=0.0300022191273473),
 Row(count_proportion=0.03537786382210251)]

In [21]:
train_prior_probabilities = [p[0] for p in train_prior_probabilities]
test_prior_probabilities = [p[0] for p in test_prior_probabilities]
sum([train_p * cv_p for train_p, cv_p in zip(train_prior_probabilities,
test_prior_probabilities)])

0.3777231597731851